In [1]:
import sys
import pathlib
cwd = pathlib.Path().cwd()
sys.path.append(cwd.parent.as_posix())
data_folder = cwd.parent.joinpath('data')

import logging
import datetime as dt
import pandas as pd

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from functools import partial

from sklearn.pipeline import make_pipeline
from sklearn.base import TransformerMixin

from telecom.transformers import Merger, TimeDifference, Clusterer, AsDummies, PurchaseRatio, ColumnsCorrector, BasicFiller
# from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from auxiliary import cv_fit, whole_fit

from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

FORMAT = '%(asctime)s > %(message)s'
logging.basicConfig(filename='fit.log', level=logging.INFO, format=FORMAT)

## load

In [2]:
n_folds = 5
bound_date = '2018-11-19'

In [3]:
# read train data
train_data = pd.read_csv(data_folder.joinpath('data_train.csv')).drop('Unnamed: 0', axis=1)
# extract required train data
train_data = train_data[train_data['buy_time'] >= dt.datetime.fromisoformat(bound_date).timestamp()]
# data/target split
data = train_data.drop('target', axis=1)
target = train_data['target']

# read compressed features
features = pd.read_csv(data_folder.joinpath('compressed_features.csv'))

# calc class weights
class_weights = dict(enumerate(compute_class_weight('balanced', classes=[0, 1], y=train_data['target'])))
# prepare folds for CV fit
folds = KFold(n_splits=n_folds, shuffle=True, random_state=29)
# prepare scoring function
f1_macro = partial(f1_score, average='macro')

## workspace

In [4]:
# prepare dataset for feature engineering
pipeline = make_pipeline(
    # Merger(features, method='backward', fillna='nearest'),
    Merger(features, method='backward', fillna='mean'),
    TimeDifference('feats_time', 'train_time'),
    Clusterer(['0', '1', '2'], n_clusters=8, random_state=13),
    
)
ft = pipeline.fit_transform(data, target)
ft['target'] = target.values
ft.head()

id  vas_id  train_time  ...  time_diff  cluster  target
6   2611143     2.0  1544994000  ...        133        4     0.0
7   3577737     5.0  1545598800  ...        168        2     0.0
9   1232876     2.0  1544994000  ...        147        2     0.0
22  1054880     1.0  1544994000  ...        105        7     0.0
24  1759351     2.0  1546203600  ...         21        4     0.0

[5 rows x 10 columns]

## data overview

In [44]:
# pipeline = make_pipeline(
#     # Merger(features, method='backward', fillna='nearest'),
#     Merger(features, method='backward', fillna='mean'),
#     # TimeDifference('feats_time', 'train_time'),
    
#     Clusterer(['0', '1', '2'], n_clusters=8, random_state=13),
#     # AsDummies(['cluster']),
#     PurchaseRatio(by=['cluster']),

#     ColumnsCorrector('drop', ['id', 'train_time', 'feats_time',]),
#     BasicFiller(strategy='mean', apply_on_fly=True),
# )
# pipeline.fit_transform(data, target)

vas_id             0             1             2  cluster  \
0          2.0  6.779413e+08 -2.634963e+08 -8.669099e+07      4.0   
1          5.0  2.525749e+08  1.176470e+09 -1.521874e+08      2.0   
2          2.0  2.518684e+08  1.178862e+09 -1.522962e+08      2.0   
3          1.0  1.936687e+08  1.252885e+09  1.229307e+09      7.0   
4          2.0  6.779413e+08 -2.634963e+08 -8.669099e+07      4.0   
...        ...           ...           ...           ...      ...   
282178     2.0  6.779413e+08 -2.634963e+08 -8.669099e+07      4.0   
282179     2.0  6.779413e+08 -2.634963e+08 -8.669099e+07      4.0   
282180     2.0  6.779413e+08 -2.634963e+08 -8.669099e+07      4.0   
282181     2.0  6.779413e+08 -2.634963e+08 -8.669099e+07      4.0   
282182     2.0 -7.989159e+08 -7.015225e+08 -1.254349e+08      1.0   

        purchase_ratio  
0             0.026010  
1             0.022268  
2             0.027333  
3             0.057143  
4             0.026010  
...                ...  
282178        0.026010  
282179        0.026010  
282180        0.026010  
282181        0.026010  
282182        0.023094  

[282183 rows x 6 columns]

## manual fit

In [5]:
# final pipeline
pipeline = make_pipeline(
    # Merger(features, method='backward', fillna='nearest'),
    Merger(features, method='backward', fillna='mean'),
    TimeDifference('feats_time', 'train_time'),
    Clusterer(['0', '1', '2'], n_clusters=8, random_state=13),
    # AsDummies(['cluster']),
    PurchaseRatio(by=['cluster']),
    ColumnsCorrector('drop', ['id', 'train_time', 'feats_time',]),
    BasicFiller(strategy='mean', apply_on_fly=True),
    
    # RandomForestClassifier(random_state=17, n_jobs=-1, class_weight=class_weights,)
    LGBMClassifier(random_state=17, n_jobs=-1, class_weight=class_weights,)
)

avg, _, _ = cv_fit(pipeline, data, target, cv=folds, scorer=f1_macro, logger=logging, prefix='[CVFIT]')
score, model = whole_fit(pipeline, data, target, scorer=f1_macro, logger=logging, prefix='[WHOLE]')

[CVFIT] Pipeline: 0.7533358612482944
[WHOLE] Pipeline: 0.75460727893761


## feature importances

In [24]:
# score, model = whole_fit(pipeline, data, target, scorer=f1_macro, logger=logging, prefix='[WHOLE]')

In [6]:
sorted_importances = sorted(normalize([model[-1].feature_importances_])[0])

fig = plt.figure(figsize=(8, 6))
plt.title(f'Feature importances [{model[-1].__class__.__name__}]')
cmap = plt.get_cmap('CMRmap')(sorted_importances)
plt.barh(model[-1].feature_name_, sorted_importances, color=cmap);

In [ ]:
#